In [2]:
import torch
from math import factorial
from pytorch3d.io import load_obj

ModuleNotFoundError: No module named 'pytorch3d'

In [ ]:
# m = load_obj("model_normalized.obj",load_textures=False)
# v_matrix = m[0]
# f_matrix = m[1].verts_idx

m = torch.load('mesh.pt')
v_matrix = m['verts']
f_matrix = m['faces']

In [3]:
# # IGNORE

# def M_ijk_torch_automatic_correction(f_matrix,v_matrix,max_m,volume_winding_test=False):
    
#     # Define face ordering
#     order = [0,1,2]
#     # Test the volume. Swap face order if needed
#     if (not volume_winding_test) and torch.all(M_ijk_torch(f_matrix,v_matrix,1,volume_winding_test=True)<0):
#         order = [0,2,1]
#         print('Corrected face ordering')

        
#     num_faces = len(f_matrix)
    
#     # Extract coordinates
#     ABC = v_matrix[f_matrix[:,torch.LongTensor(order)]]
#     # Calculate Determinants
#     dets = torch.det(ABC)
    
#     # Allocate Tensors
#     M_tensor = torch.zeros([num_faces,max_m,max_m,max_m],dtype=torch.float32)
#     C_tensor = torch.zeros([num_faces,max_m,max_m,max_m],dtype=torch.float32)
#     D_tensor = torch.zeros([num_faces,max_m,max_m,max_m],dtype=torch.float32)
#     S_tensor = torch.zeros([num_faces,max_m,max_m,max_m],dtype=torch.float32)
    
#     # Calculate C Tensor, parallellized over faces
#     for i in range(max_m):
#             for j in range(max_m):
#                 for k in range(max_m):
#                     if (i+j+k)<=max_m:
#                         C_tensor[:,i,j,k] = (ABC[:,2][:,0]**i)*(ABC[:,2][:,1]**j)*(ABC[:,2][:,2]**k)*(factorial(i+j+k)/(factorial(i)*factorial(j)*factorial(k)))
                        
#     # Calculate D Tensor, parallellized over faces
#     for i in range(max_m):
#             for j in range(max_m):
#                 for k in range(max_m):
#                     if (i+j+k)<=max_m:
#                         if (i<0) or (j<0) or (k<0):
#                             # D_ijk=0
#                             pass
#                         elif (0==i) and (0==j) and (0==k):
#                             # D_ijk = 1
#                             D_tensor[:,i,j,k] = 1
#                         else:
#                             D_tensor[:,i,j,k] = ABC[:,1][:,0]*D_tensor[:,i-1,j,k]+ABC[:,1][:,1]*D_tensor[:,i,j-1,k]+ABC[:,1][:,2]*D_tensor[:,i,j,k-1]+C_tensor[:,i,j,k]
    
    
#     # Calculate S Tensor, parallellized over faces
#     for i in range(max_m):
#             for j in range(max_m):
#                 for k in range(max_m):
#                     if (i+j+k)<=max_m:
#                         if (i<0) or (j<0) or (k<0):
#                             # S_ijk = 0
#                             pass
#                         elif (0==i) and (0==j) and (0==k):
#                             # S_ijk = 1
#                             S_tensor[:,i,j,k] = 1
#                         else:
#                             S_tensor[:,i,j,k] = ABC[:,0][:,0]*S_tensor[:,i-1,j,k]+ABC[:,0][:,1]*S_tensor[:,i,j-1,k]+ABC[:,0][:,2]*S_tensor[:,i,j,k-1]+D_tensor[:,i,j,k]
        
#     # Calculate M Tensor, parallellized over faces
#     for i in range(max_m):
#         for j in range(max_m):
#             for k in range(max_m):
#                 if (i+j+k)<=max_m:
#                     M_tensor[:,i,j,k] = ((factorial(i)*factorial(j)*factorial(k))/factorial(i+j+k+3))*(dets[:]*S_tensor[:,i,j,k])
                    
#     return torch.sum(M_tensor,0)
    

# # M_ijk_torch(torch.tensor(f_matrix,dtype=torch.long),torch.tensor(v_matrix),3)

In [1]:
def M_ijk_torch(f_matrix,v_matrix,max_m):
        
    total_num_faces = len(f_matrix)
    
    loop_batch_size = 10000
        
    m_v = torch.zeros([max_m,max_m,max_m],dtype=torch.float32)
    m_s = torch.zeros([max_m,max_m,max_m],dtype=torch.float32)
    
    for batch_num in range(ceil(total_num_faces/loop_batch_size)):
        print('calculating batch number ', batch_num)
        start = batch_num*loop_batch_size
        end = min([loop_batch_size, num_faces-start])
        num_faces = end-start

        # Extract coordinates
        ABC = v_matrix[f_matrix[start:end,:]]
        # Calculate Determinants or Cross Products
        dets = torch.det(ABC)
        cross = torch.norm(torch.cross(ABC[:,0]-ABC[:,2],ABC[:,1]-ABC[:,2]),p=2,dim = 1)


        # Allocate Tensors
        M_V_tensor = torch.zeros([num_faces,max_m,max_m,max_m],dtype=torch.float32)
        M_S_tensor = torch.zeros([num_faces,max_m,max_m,max_m],dtype=torch.float32)
        C_tensor = torch.zeros([num_faces,max_m,max_m,max_m],dtype=torch.float32)
        D_tensor = torch.zeros([num_faces,max_m,max_m,max_m],dtype=torch.float32)
        S_tensor = torch.zeros([num_faces,max_m,max_m,max_m],dtype=torch.float32)

        # Calculate C Tensor, parallellized over faces
        for i in range(max_m):
                for j in range(max_m):
                    for k in range(max_m):
                        if (i+j+k)<=max_m:
                            C_tensor[:,i,j,k] = (ABC[:,2][:,0]**i)*(ABC[:,2][:,1]**j)*(ABC[:,2][:,2]**k)*(factorial(i+j+k)/(factorial(i)*factorial(j)*factorial(k)))

        # Calculate D Tensor, parallellized over faces
        for i in range(max_m):
                for j in range(max_m):
                    for k in range(max_m):
                        if (i+j+k)<=max_m:
                            if (i<0) or (j<0) or (k<0):
                                # D_ijk=0
                                pass
                            elif (0==i) and (0==j) and (0==k):
                                # D_ijk = 1
                                D_tensor[:,i,j,k] = 1
                            else:
                                D_tensor[:,i,j,k] = ABC[:,1][:,0]*D_tensor[:,i-1,j,k]+ABC[:,1][:,1]*D_tensor[:,i,j-1,k]+ABC[:,1][:,2]*D_tensor[:,i,j,k-1]+C_tensor[:,i,j,k]


        # Calculate S Tensor, parallellized over faces
        for i in range(max_m):
                for j in range(max_m):
                    for k in range(max_m):
                        if (i+j+k)<=max_m:
                            if (i<0) or (j<0) or (k<0):
                                # S_ijk = 0
                                pass
                            elif (0==i) and (0==j) and (0==k):
                                # S_ijk = 1
                                S_tensor[:,i,j,k] = 1
                            else:
                                S_tensor[:,i,j,k] = ABC[:,0][:,0]*S_tensor[:,i-1,j,k]+ABC[:,0][:,1]*S_tensor[:,i,j-1,k]+ABC[:,0][:,2]*S_tensor[:,i,j,k-1]+D_tensor[:,i,j,k]

        # Calculate M Tensor for Volumetric moments, parallellized over faces
        for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        M_V_tensor[:,i,j,k] = ((factorial(i)*factorial(j)*factorial(k))/factorial(i+j+k+3))*(dets[:]*S_tensor[:,i,j,k])

        # Calculate M Tensor for Surface moments, parallellized over faces
        for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        M_S_tensor[:,i,j,k] = ((factorial(i)*factorial(j)*factorial(k))/factorial(i+j+k+2))*(cross[:]*S_tensor[:,i,j,k])

                    
    m_v = m_v + torch.sum(M_V_tensor,0)
    m_s = m_s + torch.sum(M_S_tensor,0)
    
    return (m_v,m_s)
    

# M_ijk_torch(torch.tensor(f_matrix,dtype=torch.long),torch.tensor(v_matrix),3)

In [5]:
def V_C_M(f_matrix,v_matrix,max_m):
    # Define face ordering
    order = [0,1,2]
    
    # Get volume and centroid
    v_c = M_ijk_torch(f_matrix,v_matrix,2)[0]
    v = torch.pow(v_c[0,0,0], 1/3)
    c = torch.zeros((1,3))
    c[0,0] = v_c[1,0,0]
    c[0,1] = v_c[0,1,0]
    c[0,2] = v_c[0,0,1]
    
    # Correct face ordering if volume is negative
    if v_c[0,0,0]<0:
        order = [0,2,1]
        v_c = M_ijk_torch(f_matrix[:,torch.LongTensor(order)],v_matrix,2)
        print('Corrected winding')
    
    # Calculate the moments
    m = M_ijk_torch(f_matrix[:,torch.LongTensor(order)],(v_matrix - c/v_c[0,0,0])/v,max_m)
    
    # Return tuple of volume, centroid, and the invariant moments
    return (v,c,m[0],m[1])
    
    
    

In [6]:
V_C_M(f_matrix,v_matrix,10)

(tensor(0.3912),
 tensor([[-1.6191e-03, -1.6944e-03, -4.1261e-05]]),
 tensor([[[ 1.0000e+00, -1.3155e-08,  6.3325e-02,  7.4882e-06,  7.6493e-03,
            2.7785e-06,  1.1426e-03,  9.1254e-07,  1.9138e-04,  2.8593e-07],
          [ 1.0128e-08, -3.2093e-06, -1.5315e-03, -9.1801e-07, -3.1739e-04,
           -4.0399e-07, -6.2335e-05, -1.4033e-07, -1.2405e-05, -4.4990e-08],
          [ 3.1138e-02, -1.3993e-06,  1.7428e-03,  4.6766e-08,  1.9432e-04,
            5.3348e-08,  2.7683e-05,  2.2866e-08,  4.5357e-06,  0.0000e+00],
          [ 1.6246e-03, -1.2036e-07, -3.8221e-05, -2.9485e-08, -1.5177e-05,
           -1.7194e-08, -3.2974e-06, -6.3451e-09,  0.0000e+00,  0.0000e+00],
          [ 2.1165e-03, -5.3903e-08,  1.0722e-04,  6.7530e-09,  1.1306e-05,
            3.4600e-09,  1.5695e-06,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 2.7469e-04,  2.2779e-08,  1.7084e-06,  1.6634e-09, -8.7179e-07,
           -8.0743e-10,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 1

In [7]:
# def VCM_loss(f_matrix_gt,v_matrix_gt,f_matrix_pred,v_matrix_pred,max_m):
#     VCM_gt   = V_C_M(f_matrix_gt  ,v_matrix_gt  ,max_m)
#     VCM_pred = V_C_M(f_matrix_pred,v_matrix_pred,max_m)
#     v_loss = torch.linalg.norm(VCM_gt[0] - VCM_pred[0])
#     c_loss = torch.linalg.norm(VCM_gt[1] - VCM_pred[1])
#     m_loss = torch.linalg.norm(VCM_gt[2] - VCM_pred[2])
    
#     return (v_loss,c_loss,m_loss)




In [8]:
# m_1 = torch.load('mesh.pt')
# v_matrix_1 = m_1['verts']
# f_matrix_1 = m_1['faces']

# # # Test against another object
# # m_2 = torch.load('mesh_2.pt')
# # v_matrix_2 = m_2['verts']
# # f_matrix_2 = m_2['faces']


# # Rotate same object and test
# v_matrix_2 = m_1['verts']
# f_matrix_2 = m_1['faces']
# theta = torch.tensor(torch.pi/1000)
# rotation_matrix = torch.tensor([[1, 0, 0], [0, torch.cos(theta), -torch.sin(theta)], [0, torch.sin(theta), torch.cos(theta)]])
# v_matrix_2 = torch.matmul( v_matrix_2 , rotation_matrix )

In [9]:
# VCM_loss(f_matrix_1,v_matrix_1,f_matrix_2,v_matrix_2,10)

In [10]:
# def VCM_loss_training(VCM_gt, f_matrix_pred, v_matrix_pred):
#     VCM_pred = V_C_M(f_matrix_pred,v_matrix_pred,VCM_gt[2].shape[0])
#     v_loss = torch.linalg.norm(VCM_gt[0] - VCM_pred[0])
#     c_loss = torch.linalg.norm(VCM_gt[1] - VCM_pred[1])
#     m_loss = torch.linalg.norm(VCM_gt[2] - VCM_pred[2])
    
#     return (v_loss,c_loss,m_loss)

In [11]:
# VCM_loss_training(V_C_M(f_matrix,v_matrix,5),f_matrix,v_matrix)

In [12]:
M_ijk_torch(f_matrix,v_matrix,5)

(tensor([[[ 5.9870e-02, -4.1261e-05,  5.8025e-04, -1.1728e-06,  1.0728e-05],
          [-1.6944e-03,  1.1383e-06, -2.1911e-05,  4.3252e-08, -4.7772e-07],
          [ 3.3326e-04, -2.3302e-07,  3.2194e-06, -6.5359e-09,  0.0000e+00],
          [-1.9756e-05,  1.3802e-08, -2.5480e-07,  0.0000e+00,  0.0000e+00],
          [ 3.7181e-06, -2.5943e-09,  0.0000e+00,  0.0000e+00,  0.0000e+00]],
 
         [[-1.6191e-03,  1.6407e-06, -1.3549e-05,  2.0269e-08, -1.8160e-07],
          [-1.1830e-04,  8.7181e-08, -7.4919e-07,  2.8516e-09,  0.0000e+00],
          [-6.3996e-06,  9.6907e-09, -3.8717e-08,  0.0000e+00,  0.0000e+00],
          [-1.0722e-06,  3.5741e-10,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [-3.9064e-08,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],
 
         [[ 2.7660e-03, -2.6677e-06,  2.5816e-05, -8.3421e-08,  0.0000e+00],
          [-1.1061e-04,  1.4503e-07, -1.2025e-06,  0.0000e+00,  0.0000e+00],
          [ 1.5220e-05, -1.8858e-08,  0.0000e+00,  0.0000e+00,  0.0000